***
# Merge data
***

In [1]:
# Create folders and getting path
import os
cwd = os.getcwd()

# Loop to create folders
folder_names = ['Dataframe', 'Output', 'Data']

folders = {}
for folder_name in folder_names:
    folders[folder_name] = os.path.join(cwd, folder_name)

    if not os.path.exists(os.path.join(cwd, folder_name)):
        os.makedirs(os.path.join(cwd, folder_name))
        print(f'Le dossier « {folder_name} » a été créé')

    else:
        print(f'Le dossier « {folder_name} » est existant')


# Création des variables de path

# Déterminer si os est win ou linux pour définir les path
if os.name == 'nt':
    slash = '\\'
elif os.name == 'posix':
    slash = '/'

path_prog =     cwd + slash
path_data =     folders['Data'] + slash
path_df =       folders['Dataframe'] + slash
path_output =   folders['Output'] + slash

# Mettre \\ pour éviter les erreurs
path_dict = [path_prog, path_data, path_df, path_output]
for path in path_dict:
    path = path.replace('\\','\\\\')


# Détermination de l'année
print()
print(path_prog)
print(path_data)
print(path_df)
print(path_output)

Le dossier « Dataframe » est existant
Le dossier « Output » est existant
Le dossier « Data » est existant

c:\Users\Charles_tour\Documents\GitHub\car_sales_forcast\
c:\Users\Charles_tour\Documents\GitHub\car_sales_forcast\Data\
c:\Users\Charles_tour\Documents\GitHub\car_sales_forcast\Dataframe\
c:\Users\Charles_tour\Documents\GitHub\car_sales_forcast\Output\


In [2]:
import pandas as pd
import numpy as np

In [44]:
# Import data
scrap_data = pd.read_hdf(path_df + 'scrap_data.h5')

vente_2017 = pd.read_hdf(path_df + 'vente_data_2017.h5')
vente_2018 = pd.read_hdf(path_df + 'vente_data_2018.h5')

In [8]:
scrap_data.head(3)

,link_model_an,year,model,brand,prix_min,prix_max,cons_min,cons_max
0,https://www.guideautoweb.com/constructeurs/acu...,2023,integra,acura,34350.0,42550.0,6.3,8.9
1,https://www.guideautoweb.com/constructeurs/acu...,2023,mdx,acura,59300.0,84400.0,9.4,13.8
2,https://www.guideautoweb.com/constructeurs/acu...,2022,mdx,acura,57900.0,83000.0,9.4,13.8


In [58]:
col_to_merge = ['brand', 'model', 'year']

scrap_data['merge_key'] = scrap_data[col_to_merge].apply(lambda x: x.str.upper()).apply(lambda x: '_'.join(x), axis=1)
scrap_data.head(3)

,link_model_an,year,model,brand,prix_min,prix_max,cons_min,cons_max,merge_key
0,https://www.guideautoweb.com/constructeurs/acu...,2023,integra,acura,34350.0,42550.0,6.3,8.9,ACURA_INTEGRA_2023
1,https://www.guideautoweb.com/constructeurs/acu...,2023,mdx,acura,59300.0,84400.0,9.4,13.8,ACURA_MDX_2023
2,https://www.guideautoweb.com/constructeurs/acu...,2022,mdx,acura,57900.0,83000.0,9.4,13.8,ACURA_MDX_2022


In [56]:
vente_2017['ANNEE_MOD'] = vente_2017['ANNEE_MOD'].astype(int)

col_to_merge = ['MARQ_VEH', 'MODEL_VEH', 'ANNEE_MOD']
vente_2017['merge_key'] = vente_2017[col_to_merge].astype(str).apply(lambda x: '*'.join(x), axis=1)
vente_2017.head(3)

,AN,NOSEQ_VEH,CLAS,TYP_VEH_CATEG_USA,MARQ_VEH,MODEL_VEH,ANNEE_MOD,MASSE_NETTE,NB_CYL,CYL_VEH,NB_ESIEU_MAX,COUL_ORIG,TYP_CARBU,TYP_DOSS_PERS,PHYS_SEX,PHYS_AGE,REG_ADM,MRC,CG_FIXE,merge_key
0,2017,2017_0000000016,PAU,AU,FORD,ESCAP,2017,1662.0,4.0,2001.0,NaN,ROU,E,P,F,76.0,Centre-du-Québec (17),Drummond (49 ),49030.0,FORD*ESCAP*2017
1,2017,2017_0000000022,PAU,AU,CHEVR,VOLT,2017,1578.0,4.0,1509.0,NaN,NOI,W,P,M,33.0,Saguenay–Lac-Saint-Jean (02),Saguenay (941),94068.0,CHEVR*VOLT*2017
2,2017,2017_0000000041,PAU,AU,NISSA,VERSA,2017,1113.0,4.0,1607.0,NaN,GRI,E,P,F,87.0,Laurentides (15),Les Laurentides (78 ),78010.0,NISSA*VERSA*2017


In [62]:
import re

In [68]:
import re

def find_matching_values(row, scrap_data):
    matches = [x for x in scrap_data['merge_key'].values if re.findall(row['merge_key'], x)]
    return matches if matches else None

# You need to pass the `vente_2017` DataFrame and the current row to the function.
match_list = vente_2017.apply(find_matching_values, axis=1, scrap_data=scrap_data)
# 75


In [71]:
match_list.head(30)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
dtype: object

In [60]:
merge_data = vente_2017.merge(scrap_data, how='left', left_on='merge_key', right_on='merge_key', indicator=True)
merge_data

,AN,NOSEQ_VEH,CLAS,TYP_VEH_CATEG_USA,MARQ_VEH,MODEL_VEH,ANNEE_MOD,MASSE_NETTE,NB_CYL,CYL_VEH,...,merge_key,link_model_an,year,model,brand,prix_min,prix_max,cons_min,cons_max,_merge
0,2017,2017_0000000016,PAU,AU,FORD,ESCAP,2017,1662.0,4.0,2001.0,...,FORD*ESCAP*2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,2017,2017_0000000022,PAU,AU,CHEVR,VOLT,2017,1578.0,4.0,1509.0,...,CHEVR*VOLT*2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,2017,2017_0000000041,PAU,AU,NISSA,VERSA,2017,1113.0,4.0,1607.0,...,NISSA*VERSA*2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,2017,2017_0000000044,PAU,AU,TOYOT,YARIS,2018,1050.0,4.0,1492.0,...,TOYOT*YARIS*2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,2017,2017_0000000071,PAU,AU,NISSA,PATHF,2017,2018.0,6.0,3509.0,...,NISSA*PATHF*2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496040,2017,2017_0006552409,PAU,AU,TOYOT,COROL,2017,1300.0,4.0,1804.0,...,TOYOT*COROL*2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
496041,2017,2017_0006552422,CAU,AU,FORD,ESCAP,2017,1623.0,4.0,1509.0,...,FORD*ESCAP*2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
496042,2017,2017_0006552429,PAU,AU,KIA,SPORT,2017,1696.0,4.0,2361.0,...,KIA*SPORT*2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
496043,2017,2017_0006552449,CAU,AU,MERCE,GLE,2018,2235.0,6.0,3001.0,...,MERCE*GLE*2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
